<a href="https://colab.research.google.com/github/dishakhanted/Facial-analytics/blob/master/facial_analytics_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import sys
sys.path.append("/content/drive/My Drive/Colab Notebooks/code")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install face_alignment

In [ ]:
use_cuda=True

In [ ]:
import numpy as np
import cv2,os
import dlib
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.backends.cudnn as cudnn
import torchvision
import torch.nn.functional as F
from PIL import Image
import datasets,hopenet, utils
from skimage import io
import face_alignment

In [ ]:
def landmark_detect(frame_name):
  #fa = face_alignment.FaceAlignment(face_alignment.LandmarksType._2D, flip_input=False)
  fa = face_alignment.FaceAlignment(face_alignment.LandmarksType._3D, flip_input=False)
  #input = io.imread('/home/semantics/2.jpg')
  input = io.imread('frame_new.jpg')
  preds = fa.get_landmarks(input)
  preds = preds[0]
  return preds

In [ ]:
if __name__ == '__main__':
    cudnn.enabled = True
    batch_size = 1
    gpu = 0
    snapshot_path = '/content/drive/My Drive/Colab Notebooks/code/frame_new.jpg'
    out_dir = '/content/drive/My Drive/Colab Notebooks/code/output/video'
    video_path='/content/drive/My Drive/Colab Notebooks/code/input1.mp4'
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    # if not os.path.exists(args.video_path):
    #     sys.exit('Video does not exist')

    # ResNet50 structure
    model = hopenet.Hopenet(torchvision.models.resnet.Bottleneck, [3, 4, 6, 3], 66)
    face_model='/content/drive/My Drive/Colab Notebooks/code/mmod_human_face_detector.dat'
    # Dlib face detection model
    cnn_face_detector = dlib.cnn_face_detection_model_v1(face_model)

    print ('Loading snapshot.')
    # Load snapshot
   # saved_state_dict = torch.load(snapshot_path)
    #model.load_state_dict(saved_state_dict)

    print ('Loading data.')

    transformations = transforms.Compose([transforms.Scale(224),
    transforms.CenterCrop(224), transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

    model.cuda(gpu)

    print ('Ready to test network.')

    # Test the Model
    model.eval()  # Change model to 'eval' mode (BN uses moving mean/var).
    total = 0
    idx_tensor = [idx for idx in range(66)]
    idx_tensor = torch.FloatTensor(idx_tensor).cuda(gpu)
    video = cv2.VideoCapture(video_path)
    # New cv2
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))   # float
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT)) # float

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    out = cv2.VideoWriter('/content/drive/My Drive/Colab Notebooks/code/videof.txt.avi', fourcc, 1, (width, height))

    # # Old cv2
    # width = int(video.get(cv2.cv.CV_CAP_PROP_FRAME_WIDTH))   # float
    # height = int(video.get(cv2.cv.CV_CAP_PROP_FRAME_HEIGHT)) # float
    #
    # # Define the codec and create VideoWriter object
    # fourcc = cv2.cv.CV_FOURCC(*'MJPG')
    # out = cv2.VideoWriter('output/video/output-%s.avi' % args.output_string, fourcc, 30.0, (width, height))

    txt_out = open('/content/drive/My Drive/Colab Notebooks/code/videof.txt', 'w')

    #from moviepy.editor import VideoFileClip
    #clip = VideoFileClip("Video_path")
    #print( clip.duration )

    frame_num=1

    while frame_num <= 100:
        print ("frame num",frame_num)
        #cv2.imwrite("output/"+str(frame_num)+".jpg", frame)
        cv2.imwrite("frame_new.jpg", frame)
        ret,frame = video.read()
        if ret == False:
            break


        #cv2.imwrite("/content/drive/My Drive/Colab Notebooks/code/output/"+str(frame_num)+".jpg", frame)
        

        cv2_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
#	cv2.imwrite("cnn_face_detection.png", cv2_frame)
#	cv2_frame = cv2.imread("cnn_face_detection.png")
        # Dlib detect
        dets = cnn_face_detector(cv2_frame, 1)

        for idx, det in enumerate(dets):
            # Get x_min, y_min, x_max, y_max, conf
            x_min = det.rect.left()
            y_min = det.rect.top()
            x_max = det.rect.right()
            y_max = det.rect.bottom()
            conf = det.confidence

            if conf > 1.0:
                print('yes')
                bbox_width = abs(x_max - x_min)
                bbox_height = abs(y_max - y_min)
                x_min -= 2 * bbox_width / 4
                x_max += 2 * bbox_width / 4
                y_min -= 3 * bbox_height / 4
                y_max += bbox_height / 4
                x_min = max(x_min, 0); y_min = max(y_min, 0)
                x_max = min(frame.shape[1], x_max); y_max = min(frame.shape[0], y_max)
                x_min=int(x_min)
                x_max=int(x_max)
                y_min=int(y_min)
                y_max=int(y_max)
                # Crop image
                img = cv2_frame[y_min:y_max,x_min:x_max]
                img = Image.fromarray(img)

                # Transform
                img = transformations(img)
                img_shape = img.size()
                img = img.view(1, img_shape[0], img_shape[1], img_shape[2])
                img = Variable(img).cuda(gpu)

                yaw, pitch, roll = model(img)

                yaw_predicted = F.softmax(yaw)
                pitch_predicted = F.softmax(pitch)
                roll_predicted = F.softmax(roll)
                # Get continuous predictions in degrees.
                yaw_predicted = torch.sum(yaw_predicted.data[0] * idx_tensor) * 3 - 99
                pitch_predicted = torch.sum(pitch_predicted.data[0] * idx_tensor) * 3 - 99
                roll_predicted = torch.sum(roll_predicted.data[0] * idx_tensor) * 3 - 99

                # Print new frame with cube and axis
                txt_out.write(str(frame_num) + ' %f %f %f\n' % (yaw_predicted, pitch_predicted, roll_predicted))
                # utils.plot_pose_cube(frame, yaw_predicted, pitch_predicted, roll_predicted, (x_min + x_max) / 2, (y_min + y_max) / 2, size = bbox_width)
                utils.draw_axis(frame, yaw_predicted, pitch_predicted, roll_predicted, tdx = (x_min + x_max) / 2, tdy= (y_min + y_max) / 2, size = bbox_height/2)
#                print ("writing frame",frame_num)
                cv2.imwrite("/content/drive/My Drive/Colab Notebooks/code/output/output"+str(frame_num)+".jpg", frame)

                # Plot expanded bounding box
                # cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0,255,0), 1)
                landmark_coordinates = landmark_detect(frame_num)
                res = {}
                res['frame_name']=str(frame_num)
                res['head_rotation']={}
                res['head_rotation']['yaw'] = float(yaw_predicted)
                res['head_rotation']['pitch'] = float(pitch_predicted)
                res['head_rotation']['roll'] = float(roll_predicted)
                res['facial_landmarks']= landmark_coordinates.tolist()
                print(res['facial_landmarks'])
                with open('./output/combined_'+str(frame_num)+'.json','w') as fitr:
                  json.dump(res,fitr)
                # Plot expanded bounding box
                #cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0,255,0), 1)
                
        out.write(frame)
        frame_num += 1
    out.release()
    video.release()

Loading snapshot.
Loading data.
Ready to test network.
frame num 1


/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:211: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


frame num 2
frame num 3
frame num 4
frame num 5
frame num 6
frame num 7
frame num 8
frame num 9
frame num 10
frame num 11
frame num 12
frame num 13
frame num 14
frame num 15
frame num 16
frame num 17
frame num 18
frame num 19
frame num 20
frame num 21
frame num 22
frame num 23
frame num 24
frame num 25
frame num 26
frame num 27
frame num 28
frame num 29
frame num 30
frame num 31
frame num 32
frame num 33
frame num 34
frame num 35
frame num 36
frame num 37
frame num 38
frame num 39
frame num 40
frame num 41
frame num 42
frame num 43
frame num 44
frame num 45
frame num 46
frame num 47
frame num 48
frame num 49
frame num 50
frame num 51
frame num 52
frame num 53
frame num 54
frame num 55
frame num 56
frame num 57
frame num 58
frame num 59
frame num 60
frame num 61
frame num 62
frame num 63
frame num 64
frame num 65
frame num 66
frame num 67
frame num 68
frame num 69
frame num 70
frame num 71
frame num 72
frame num 73
frame num 74
frame num 75
frame num 76
frame num 77
frame num 78
frame n

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:120: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:121: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:122: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[[1088.0, 297.0, -214.11422729492188], [1033.0, 312.0, -169.9148712158203], [978.0, 336.0, -128.4468231201172], [931.0, 352.0, -87.6018295288086], [884.0, 375.0, -36.206092834472656], [861.0, 430.0, 14.909685134887695], [861.0, 485.0, 56.65422821044922], [861.0, 540.0, 98.76904296875], [868.0, 618.0, 121.3671875], [868.0, 681.0, 103.79692840576172], [868.0, 713.0, 65.97899627685547], [884.0, 744.0, 19.433454513549805], [915.0, 768.0, -34.01453399658203], [963.0, 775.0, -81.75713348388672], [1018.0, 783.0, -119.06302642822266], [1080.0, 791.0, -156.4366912841797], [1151.0, 799.0, -198.46510314941406], [1222.0, 407.0, -165.5641632080078], [1253.0, 430.0, -162.4379425048828], [1277.0, 469.0, -156.66278076171875], [1277.0, 501.0, -148.40000915527344], [1277.0, 532.0, -140.37669372558594], [1300.0, 666.0, -131.22946166992188], [1308.0, 697.0, -138.44168090820312], [1300.0, 728.0, -144.4312744140625], [1292.0, 760.0, -149.11968994140625], [1253.0, 783.0, -150.36471557617188], [1222.0, 603.0,

FileNotFoundError: ignored

In [ ]:
model = hopenet.Hopenet(torchvision.models.resnet.Bottleneck, [3, 4, 6, 3], 66)
face_model='/content/drive/My Drive/Colab Notebooks/code/mmod_human_face_detector.dat'


In [ ]:
cnn_face_detector = dlib.cnn_face_detection_model_v1(face_model)
